<a href="https://colab.research.google.com/github/engineereliab076/NATURAL-LANGUAGE-PROCESSING/blob/main/BERTonNER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets seqeval


In [ ]:
#login in to the huggingface hub
from huggingface_hub import notebook_login

notebook_login()


In [ ]:
import numpy as np
import pandas as pd

DATA LOADING AND PREPROCESSING

In [ ]:
# load dataset

from datasets import load_dataset

ds = load_dataset("conll2003")
print(ds)

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})


In [ ]:
# tokenize the data

from transformers import AutoTokenizer

#load model
checkpoint = "bert-base-cased"

# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

label_list = ds["train"].features["ner_tags"].feature.names

#tokenize function

def tokenize_fxn(examples):
  tokenized_inputs = tokenizer(examples["tokens"], truncation= True, is_split_into_words=True)
  labels = []

  for i, label in enumerate(examples["ner_tags"]):
    word_ids = tokenized_inputs.word_ids(batch_index=i)
    label_ids = [-100 if word_id is None else label[word_id] for word_id in word_ids]
    labels.append(label_ids)
  tokenized_inputs["labels"] = labels
  return tokenized_inputs

tokenized_datasets = ds.map(tokenize_fxn, batched=True)




Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [ ]:
#set up data collator

from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

MODEL TRAINING  

In [ ]:
#LOAD pretrained model

from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(checkpoint, num_labels=len(label_list))

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# training arguments

from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./bert-ner",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=4,
    weight_decay=0.01,
    push_to_hub=True,
    hub_model_id="Engineereliab076/bert-ner",

)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# initialize trainer

from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)



<ipython-input-27-5078da9e171a>:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# train the model

trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: engineereliab076 (engineereliab076-rayat-bahra-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss
1,0.082000,0.079563
2,0.039900,0.072747
3,0.024100,0.067739
4,0.013700,0.068485


TrainOutput(global_step=7024, training_loss=0.058229331970893715, metrics={'train_runtime': 853.8526, 'train_samples_per_second': 65.777, 'train_steps_per_second': 8.226, 'total_flos': 1228413887136198.0, 'train_loss': 0.058229331970893715, 'epoch': 4.0})

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.5 MB/s eta 0:00:00


In [ ]:
#evaluate
from evaluate import load

# Load the seqeval metric for NER tasks
metric = load("seqeval")

results = trainer.evaluate()

print(results)

# Get predictions on the validation set
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])

# Convert predictions to label IDs
predictions = predictions.argmax(axis=-1)

# Align predictions and labels to their original form
true_labels = [[label_list[l] for l in label if l != -100] for label in labels]
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

# Compute and display the metrics
results = metric.compute(predictions=true_predictions, references=true_labels)
print(results)


{'eval_loss': 0.06848490238189697, 'eval_runtime': 8.9374, 'eval_samples_per_second': 363.639, 'eval_steps_per_second': 45.539, 'epoch': 4.0}
{'LOC': {'precision': 0.972338642078793, 'recall': 0.9573590096286108, 'f1': 0.9647906847795953, 'number': 3635}, 'MISC': {'precision': 0.8455128205128205, 'recall': 0.8912162162162162, 'f1': 0.8677631578947368, 'number': 1480}, 'ORG': {'precision': 0.9215900802334063, 'recall': 0.9352331606217616, 'f1': 0.9283614988978691, 'number': 2702}, 'PER': {'precision': 0.9674432497013142, 'recall': 0.9729648543106038, 'f1': 0.9701961958963607, 'number': 3329}, 'overall_precision': 0.9408673969186927, 'overall_recall': 0.9478736766553023, 'overall_f1': 0.9443575418994413, 'overall_accuracy': 0.9860775887443339}


In [ ]:
trainer.push_to_hub()


CommitInfo(commit_url='https://huggingface.co/Engineereliab076/bert-ner/commit/0c24671ba87b4fe18f113ce65da9c601c605f6f3', commit_message='End of training', commit_description='', oid='0c24671ba87b4fe18f113ce65da9c601c605f6f3', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Engineereliab076/bert-ner', endpoint='https://huggingface.co', repo_type='model', repo_id='Engineereliab076/bert-ner'), pr_revision=None, pr_num=None)